In [8]:
"""
╔══════════════════════════════════════════════════════════════════════╗
║  HEDGING FRAMEWORK — SCR-TARGET BACKTEST  (PATCHED v3)             ║
║                                                                     ║
║  Fixes applied:                                                     ║
║    #2  Shared mutable SCRAggregator — snapshot/restore per instr.  ║
║    #3  scr_after_hedge uses current_nb instead of initial nb       ║
║    #4  MV_EQUITY_T0 unified with PORTFOLIO equity sleeve           ║
║    #9  Daily monitoring uses instrument-specific IV surface        ║
║   #10  Notional cap: MAX_NOTIONAL_PCT limits nb to a fraction      ║
║        of the equity sleeve MV, applied at initial sizing and      ║
║        daily rebalancing                                            ║
║                                                                     ║
║  Reporting:                                                         ║
║    Integrated plots_diversified.py 20-chart PDF report             ║
║                                                                     ║
║  Usage:  python hedging_backtest_fixed.py                           ║
╚══════════════════════════════════════════════════════════════════════╝

Sections:
    0. Configuration
    1. Data Loading
    2. Black-Scholes Pricing
    3. Capital Model (Solvency II Equity)
    4. SCR Aggregator (Diversified Portfolio)
    5. Roll Schedulers (Systematic + Signal-Driven)
    6. SCR-Target Strategy (Core Engine)
    7. Aggregation & Distribution
    8. Reporting (20-Chart PDF — from plots_diversified.py)
    9. Main Pipeline
"""

import os, sys, time, warnings, copy
import numpy as np
import pandas as pd
from dataclasses import dataclass, field
from typing import List, Tuple, Optional, Dict, Any
from scipy.stats import norm

warnings.filterwarnings('ignore')

# ═══════════════════════════════════════════════════════════════════════
# §0  CONFIGURATION
# ═══════════════════════════════════════════════════════════════════════

# ── Data ──────────────────────────────────────────────────────────────
DATA_PATH = r"C:\\Users\\jules\\Desktop\\Hedging Framework - Thesis\\Input\\SPX_IV_Cleaned.csv"
CSV_SEP = ';'
CSV_DECIMAL = ','
CSV_ENCODING = 'utf-8-sig'
CSV_DATE_FMT = '%d/%m/%Y'

# ── Backtest window ───────────────────────────────────────────────────
START_DATE = '2006-06-30'
END_DATE   = '2025-12-30'
N_STARTS   = 5

# ── FIX #4: Unified equity MV ────────────────────────────────────────
PORTFOLIO_TOTAL_MV = 500_000_000
EQUITY_WEIGHT      = 0.40
MV_EQUITY_T0       = int(PORTFOLIO_TOTAL_MV * EQUITY_WEIGHT)  # = 200_000_000

# ── SCR Target parameters ────────────────────────────────────────────
TARGET_COVERAGE = 0.30
BAND_LOW        = 0.25
BAND_HIGH       = 0.35

# ── Notional cap ─────────────────────────────────────────────────────
# Maximum notional covered by puts as a fraction of the equity sleeve MV.
# e.g. 1.0 = 100% (no cap), 0.50 = 50%, None = disabled (same as 1.0)
MAX_NOTIONAL_PCT = None

# ── Portfolio (multi-asset for diversification) ───────────────────────
SCR_OTHER_MODULES  = {
    'interest_rate': 10_000_000,
    'spread':         7_000_000,
    'currency':      12_000_000,
}
RATE_SCENARIO = 'down'

# ── Instrument grid ───────────────────────────────────────────────────
MONEYNESS_FILTER = [90, 100]
MATURITY_FILTER  = []

# ── Roll logic ────────────────────────────────────────────────────────
ROLL_MODE = 'systematic'

SIGNAL_COLUMN    = 'signal'
SIGNAL_THRESHOLD = 1.0
SIGNAL_MODE      = 'above'
SIGNAL_HOLD_DAYS = None
SIGNAL_COOLDOWN  = 5

# ── Output ────────────────────────────────────────────────────────────
OUTPUT_FOLDER = f"Output_SCRTarget_{int(TARGET_COVERAGE*100)}pct"
FOCUS_MONEYS  = [90, 100]
FOCUS_MATS    = [30, 90, 180, 360, 540]


# ═══════════════════════════════════════════════════════════════════════
# §1  DATA LOADING
# ═══════════════════════════════════════════════════════════════════════

def compute_symmetric_adjustment(spot_series, window=756, floor=-0.10,
                                  cap=0.10, base_charge=0.39):
    """SII Symmetric Adjustment: SA = 0.5*(S - MA_3y)/MA_3y, clipped."""
    ma = spot_series.rolling(window, min_periods=window).mean()
    sa_raw = 0.5 * (spot_series - ma) / ma
    return sa_raw.clip(floor, cap).dropna()

def load_data(filepath):
    """Load IV surface CSV and return (df_raw, spot_series, sa_series, dates)."""
    print(f"Loading: {filepath}")
    df = pd.read_csv(filepath, sep=CSV_SEP, decimal=CSV_DECIMAL,
                     encoding=CSV_ENCODING)
    df['Date'] = pd.to_datetime(df['Date'], format=CSV_DATE_FMT)

    rename = {'sigma': 'iv', 'dividend yield': 'q', 'risk-free rates': 'r'}
    df.rename(columns=rename, inplace=True)

    for col in ('iv', 'q', 'r'):
        if col in df.columns and df[col].mean() > 1:
            df[col] = df[col] / 100.0

    df['tau'] = df['maturity'] / 365.0

    spot_df = df[['Date','spot']].drop_duplicates('Date').sort_values('Date')
    spot_series = spot_df.set_index('Date')['spot']

    sa = compute_symmetric_adjustment(spot_series)
    sa_map = sa.to_dict()
    valid = set(sa.index)
    df = df[df['Date'].isin(valid)].copy()
    df['SA'] = df['Date'].map(sa_map)
    df['equity_shock'] = 0.39 + df['SA']

    dates = sorted(df['Date'].unique())
    n_instr = df.groupby(['moneyness','maturity']).ngroups
    print(f"  {dates[0].date()} -> {dates[-1].date()} | "
          f"{len(dates)} days | {n_instr} instruments")
    return df, spot_series, sa, dates


# ═══════════════════════════════════════════════════════════════════════
# §2  BLACK-SCHOLES PRICING
# ═══════════════════════════════════════════════════════════════════════

def _d1d2(S, K, tau, r, q, sigma):
    tau = np.maximum(tau, 1e-10); st = np.sqrt(tau)
    d1 = (np.log(S/K) + (r-q+0.5*sigma**2)*tau) / (sigma*st)
    return d1, d1 - sigma*st

def bs_put_price(S, K, tau, r, q, sigma):
    d1, d2 = _d1d2(S, K, tau, r, q, sigma)
    return K*np.exp(-r*tau)*norm.cdf(-d2) - S*np.exp(-q*tau)*norm.cdf(-d1)

def forward_price(S, r, q, tau):
    return S * np.exp((r-q)*tau)


# ═══════════════════════════════════════════════════════════════════════
# §3  CAPITAL MODEL — SOLVENCY II EQUITY
# ═══════════════════════════════════════════════════════════════════════

class SolvencyIIEquity:
    """SCR Equity Type 1: shock = 39% + SA(t), SA in [-10%, +10%]."""
    def __init__(self, base_charge=0.39):
        self.base_charge = base_charge
    def get_shock(self, market_row):
        return market_row.get('equity_shock', self.base_charge)
    def scr_before_hedge(self, mv, market_row):
        return mv * self.get_shock(market_row)
    def scr_after_hedge(self, mv, market_row, nb, hg):
        return max(0.0, self.scr_before_hedge(mv, market_row) - nb * hg)


# ═══════════════════════════════════════════════════════════════════════
# §4  SCR AGGREGATOR — DIVERSIFIED PORTFOLIO
# ═══════════════════════════════════════════════════════════════════════

SCR_MODULES = ['interest_rate','equity','property','spread','currency','concentration']

_CORR_DOWN = np.array([
    [1.00, 0.50, 0.50, 0.50, 0.25, 0.00],
    [0.50, 1.00, 0.75, 0.75, 0.25, 0.00],
    [0.50, 0.75, 1.00, 0.50, 0.25, 0.00],
    [0.50, 0.75, 0.50, 1.00, 0.25, 0.00],
    [0.25, 0.25, 0.25, 0.25, 1.00, 0.00],
    [0.00, 0.00, 0.00, 0.00, 0.00, 1.00],
])
_CORR_UP = np.array([
    [1.00, 0.00, 0.00, 0.00, 0.25, 0.00],
    [0.00, 1.00, 0.75, 0.75, 0.25, 0.00],
    [0.00, 0.75, 1.00, 0.50, 0.25, 0.00],
    [0.00, 0.75, 0.50, 1.00, 0.25, 0.00],
    [0.25, 0.25, 0.25, 0.25, 1.00, 0.00],
    [0.00, 0.00, 0.00, 0.00, 0.00, 1.00],
])

class SCRAggregator:
    """SII correlation-based SCR Market aggregation with marginal factors."""
    def __init__(self, scenario='down'):
        self.corr = _CORR_DOWN.copy() if scenario == 'down' else _CORR_UP.copy()
        self._idx = {m: i for i, m in enumerate(SCR_MODULES)}
        self._scr = {m: 0.0 for m in SCR_MODULES}

    def set_scr(self, mod, val): self._scr[mod] = max(0.0, val)
    def get_scr(self, mod): return self._scr.get(mod, 0.0)

    # FIX #2: Snapshot/restore
    def snapshot(self):
        return self._scr.copy()
    def restore(self, state):
        self._scr = state.copy()

    @property
    def _vec(self):
        return np.array([self._scr[m] for m in SCR_MODULES])

    def scr_market(self):
        v = self._vec; return float(np.sqrt(v @ self.corr @ v))

    def marginal_factor(self, mod):
        total = self.scr_market()
        if total <= 0: return 1.0
        i = self._idx[mod]; v = self._vec
        return float(self.corr[i, :] @ v) / total

    def scr_market_with_hedge(self, mod, saving):
        orig = self._scr[mod]
        self._scr[mod] = max(0.0, orig - saving)
        result = self.scr_market()
        self._scr[mod] = orig
        return result

    def summary(self):
        lines = [f"SCR Market Aggregation", f"{'─'*50}"]
        for m in SCR_MODULES:
            v = self._scr[m]
            if v > 0:
                lines.append(f"  {m:<18} {v:>14,.0f}   MF={self.marginal_factor(m):.3f}")
        lines.append(f"  {'─'*50}")
        lines.append(f"  {'SCR Market':<18} {self.scr_market():>14,.0f}")
        lines.append(f"  {'Sum (undiv)':<18} {sum(self._scr.values()):>14,.0f}")
        div = sum(self._scr.values()) - self.scr_market()
        lines.append(f"  {'Divers. benefit':<18} {div:>14,.0f}")
        return '\n'.join(lines)


# ═══════════════════════════════════════════════════════════════════════
# §5  ROLL SCHEDULERS
# ═══════════════════════════════════════════════════════════════════════

class SystematicRoll:
    def generate_schedule(self, dates, maturity_days, **kw):
        n = len(dates); freq = int(maturity_days * 252/365)
        schedule, i = [], 0
        while i < n:
            j = min(i + freq, n - 1)
            if j > i: schedule.append((i, j))
            i += freq
        return schedule

class SignalDrivenRoll:
    def __init__(self, signal_series, threshold=1.0, mode='above',
                 hold_days=None, cooldown=5):
        self.signal = signal_series
        self.threshold = threshold
        self.mode = mode
        self.hold_days = hold_days
        self.cooldown = cooldown

    def _triggered(self, date, prev_date=None):
        if date not in self.signal.index: return False
        val = self.signal.loc[date]
        if self.mode == 'above': return val > self.threshold
        if self.mode == 'below': return val < self.threshold
        if self.mode == 'cross_above':
            if prev_date is None or prev_date not in self.signal.index: return False
            return self.signal.loc[prev_date] <= self.threshold < val
        if self.mode == 'cross_below':
            if prev_date is None or prev_date not in self.signal.index: return False
            return self.signal.loc[prev_date] >= self.threshold > val
        return False

    def generate_schedule(self, dates, maturity_days, **kw):
        n = len(dates)
        hold = self.hold_days or int(maturity_days * 252/365)
        schedule, next_ok = [], 0
        for i in range(n):
            if i < next_ok: continue
            prev = dates[i-1] if i > 0 else None
            if self._triggered(dates[i], prev):
                j = min(i + hold, n - 1)
                if j > i: schedule.append((i, j))
                next_ok = j + self.cooldown
        return schedule


# ═══════════════════════════════════════════════════════════════════════
# §6  SCR-TARGET STRATEGY
# ═══════════════════════════════════════════════════════════════════════

@dataclass
class TradeRecord:
    instrument_id: str; instrument_type: str
    date_buy: Any; date_exp: Any
    underlying_buy: float; underlying_exp: float; strike: float
    nb_contracts: float; premium_paid_usd: float; budget_allocated_usd: float
    payoff_usd: float; pnl_usd: float; pnl_pct_mv: float
    scr_before: float; scr_after: float; scr_saving_gross: float
    net_capital_benefit: float; M_net_pct: float
    extras: Dict = field(default_factory=dict)

    def to_dict(self):
        d = {k: v for k, v in self.__dict__.items() if k != 'extras'}
        if self.extras: d.update(self.extras)
        return d


def run_scr_target_roll(
    df_instrument, df_daily_instr, schedule,
    moneyness, maturity_days, capital_model, aggregator,
    spot_t0, mv_equity_t0,
    target=0.30, band_lo=0.25, band_hi=0.35, scr_module='equity',
    max_notional_pct=None,
):
    """Execute SCR-target strategy for one instrument over one backtest window.
    FIX #9: df_daily_instr is instrument-specific.
    max_notional_pct: if set, caps nb so that nb*S <= max_notional_pct * MV_equity."""
    results = []
    for idx_buy, idx_exp in schedule:
        row_buy = df_instrument.iloc[idx_buy].to_dict()
        row_exp = df_instrument.iloc[idx_exp].to_dict()
        record = _execute_single_roll(
            row_buy, row_exp, df_daily_instr, moneyness, maturity_days,
            capital_model, aggregator, spot_t0, mv_equity_t0,
            target, band_lo, band_hi, scr_module, max_notional_pct)
        if record is not None:
            results.append(record)
    return results


def _execute_single_roll(
    row_buy, row_exp, df_daily_instr, moneyness, maturity_days,
    capital, agg, spot_t0, mv_t0,
    target, band_lo, band_hi, scr_module, max_notional_pct=None,
):
    S_buy = row_buy['spot']
    date_buy, date_exp = row_buy['Date'], row_exp['Date']
    MV_eq = mv_t0 * (S_buy / spot_t0)

    tau_buy = row_buy['tau']
    F = forward_price(S_buy, row_buy['r'], row_buy['q'], tau_buy)
    K = (moneyness / 100.0) * F
    unit_price = bs_put_price(S_buy, K, tau_buy, row_buy['r'], row_buy['q'], row_buy['iv'])
    if unit_price <= 0 or not np.isfinite(unit_price): return None

    shock = capital.get_shock(row_buy)
    S_shocked = S_buy * (1 - shock)
    hg = bs_put_price(S_shocked, K, tau_buy, row_buy['r'], row_buy['q'], row_buy['iv']) - unit_price
    if hg <= 0: return None

    scr_before = capital.scr_before_hedge(MV_eq, row_buy)

    if agg is not None:
        scr_mkt = agg.scr_market()
        mf = agg.marginal_factor(scr_module)
        eff_gain = hg * mf
        nb = (target * scr_mkt) / eff_gain if eff_gain > 0 else 0
    else:
        mf = 1.0; scr_mkt = scr_before
        nb = (target * scr_before) / hg

    # ── Notional cap on initial sizing ────────────────────────────
    nb_uncapped = nb
    notional_capped = False
    if max_notional_pct is not None and S_buy > 0:
        max_nb = (max_notional_pct * MV_eq) / S_buy
        if nb > max_nb:
            nb = max_nb
            notional_capped = True

    initial_premium = nb * unit_price
    current_nb = nb

    # ── Daily monitoring (FIX #9: instrument-specific) ────────────
    adjustments = []
    total_adj_cost = 0.0
    mask = (df_daily_instr.index > date_buy) & (df_daily_instr.index < date_exp)
    daily_dates = df_daily_instr.index[mask]
    daily_cov = [(date_buy, target, nb)]

    for day in daily_dates:
        dr = df_daily_instr.loc[day]
        if isinstance(dr, pd.DataFrame): dr = dr.iloc[0]
        dd = dr.to_dict(); dd['Date'] = day
        S_d = dd.get('spot', S_buy)
        iv_d = dd.get('iv', row_buy.get('iv', 0.20))
        r_d = dd.get('r', row_buy.get('r', 0.03))
        q_d = dd.get('q', row_buy.get('q', 0.015))
        days_el = (day - date_buy).days
        tau_rem = max(1e-10, (maturity_days - days_el) / 365.0)

        MV_d = mv_t0 * (S_d / spot_t0)
        shock_d = dd.get('equity_shock', capital.get_shock(dd))
        scr_eq_d = MV_d * shock_d

        S_sh_d = S_d * (1 - shock_d)
        put_cur = bs_put_price(S_d, K, tau_rem, r_d, q_d, iv_d)
        put_sh  = bs_put_price(S_sh_d, K, tau_rem, r_d, q_d, iv_d)
        hg_d = put_sh - put_cur

        if hg_d <= 0 or scr_eq_d <= 0:
            daily_cov.append((day, 0.0, current_nb)); continue

        if agg is not None:
            orig_eq = agg.get_scr(scr_module)
            agg.set_scr(scr_module, scr_eq_d)
            scr_mkt_d = agg.scr_market()
            mf_d = agg.marginal_factor(scr_module)
            cov = (current_nb * hg_d * mf_d) / scr_mkt_d if scr_mkt_d > 0 else 0
        else:
            mf_d = 1.0; scr_mkt_d = scr_eq_d
            cov = (current_nb * hg_d) / scr_eq_d

        if cov < band_lo or cov > band_hi:
            eff_g = hg_d * mf_d if agg else hg_d
            nb_tgt = (target * scr_mkt_d) / eff_g if eff_g > 0 else current_nb
            # ── Notional cap on rebalance ─────────────────────────
            if max_notional_pct is not None and S_d > 0:
                max_nb_d = (max_notional_pct * MV_d) / S_d
                if nb_tgt > max_nb_d:
                    nb_tgt = max_nb_d
                    notional_capped = True
            delta = nb_tgt - current_nb
            adj_cost = delta * put_cur
            adjustments.append({
                'date': day, 'direction': 'buy' if delta > 0 else 'sell',
                'delta_nb': delta, 'price_mtm': put_cur, 'cost': adj_cost,
                'cov_before': cov, 'spot': S_d, 'iv': iv_d})
            total_adj_cost += adj_cost
            current_nb = nb_tgt
            cov = target

        daily_cov.append((day, cov, current_nb))
        if agg is not None: agg.set_scr(scr_module, orig_eq)

    # ── Expiry (FIX #3: use current_nb and expiry conditions) ────
    days_pass = (date_exp - date_buy).days
    tau_f = max(0, (maturity_days - days_pass) / 365.0)
    S_exp = row_exp['spot']
    if tau_f > 2/365:
        payoff_u = bs_put_price(S_exp, K, tau_f, row_exp['r'], row_exp['q'], row_exp['iv'])
    else:
        payoff_u = max(K - S_exp, 0)
    payoff_tot = current_nb * payoff_u
    total_prem = initial_premium + total_adj_cost
    pnl = payoff_tot - total_prem
    pnl_pct = pnl / MV_eq * 100 if MV_eq > 0 else 0

    shock_exp = capital.get_shock(row_exp)
    S_shocked_exp = S_exp * (1 - shock_exp)
    MV_exp = mv_t0 * (S_exp / spot_t0)
    if tau_f > 2/365:
        hg_exp = bs_put_price(S_shocked_exp, K, tau_f, row_exp['r'], row_exp['q'], row_exp['iv']) \
                 - bs_put_price(S_exp, K, tau_f, row_exp['r'], row_exp['q'], row_exp['iv'])
    else:
        hg_exp = max(K - S_shocked_exp, 0) - max(K - S_exp, 0)
    scr_before_exp = capital.scr_before_hedge(MV_exp, row_exp)
    scr_after = capital.scr_after_hedge(MV_exp, row_exp, current_nb, hg_exp)
    scr_saving = scr_before_exp - scr_after

    if agg is not None:
        scr_mkt_before = agg.scr_market()
        scr_mkt_after = agg.scr_market_with_hedge(scr_module, scr_saving)
        scr_saving_div = scr_mkt_before - scr_mkt_after
    else:
        scr_saving_div = scr_saving

    cov_vals = [c[1] for c in daily_cov]
    cov_mean = np.mean(cov_vals) if cov_vals else target

    scr_gain_pu = hg / S_buy if S_buy else 0
    cost_pu = total_prem / (current_nb * S_buy) if current_nb > 0 and S_buy > 0 else 0
    pnl_pu = (payoff_u - total_prem / current_nb) / S_buy if current_nb > 0 and S_buy > 0 else 0

    scr_gain_pb_div = scr_saving_div / total_prem if total_prem > 0 else 0
    scr_net_pb_div = (scr_saving_div - total_prem) / total_prem if total_prem > 0 else 0

    extras = {
        'moneyness': moneyness, 'maturity': maturity_days,
        'iv_buy': row_buy.get('iv'), 'equity_shock': row_buy.get('equity_shock'),
        'return_underlying': (S_exp / S_buy - 1) * 100 if S_buy else 0,
        'put_price_pct_spot': unit_price / S_buy * 100 if S_buy else 0,
        'strategy_type': 'scr_target', 'diversified_mode': agg is not None,
        'marginal_factor': mf,
        'MV_equity': MV_eq, 'hedge_gain_per_contract': hg,
        'unit_price': unit_price, 'payoff_per_unit': payoff_u,
        'nb_contracts_initial': nb, 'nb_contracts_final': current_nb,
        'nb_contracts': current_nb,
        'initial_premium': initial_premium,
        'adjustment_cost': total_adj_cost,
        'total_premium': total_prem,
        'premium_pct_mv': total_prem / MV_eq * 100 if MV_eq else 0,
        'SCR_nu_pct': scr_before / MV_eq * 100 if MV_eq else 0,
        'SCR_hedged_pct': scr_after / MV_exp * 100 if MV_exp else 0,
        'gross_scr_saving_pct': scr_saving / MV_exp * 100 if MV_exp else 0,
        'scr_coverage_initial': target,
        'scr_coverage_mean': cov_mean,
        'scr_coverage_min': np.min(cov_vals) if cov_vals else target,
        'scr_coverage_max': np.max(cov_vals) if cov_vals else target,
        'n_adjustments': len(adjustments),
        'n_adj_buy': sum(1 for a in adjustments if a['direction']=='buy'),
        'n_adj_sell': sum(1 for a in adjustments if a['direction']=='sell'),
        'scr_gain_pu_iso': scr_gain_pu,
        'scr_gain_pu_div': scr_gain_pu * mf,
        'cost_pu': cost_pu,
        'scr_net_pu_iso': scr_gain_pu - cost_pu,
        'scr_net_pu_div': scr_gain_pu * mf - cost_pu,
        'pnl_pu': pnl_pu,
        'efficiency_pu_iso': (scr_gain_pu / cost_pu) if cost_pu > 0 else 0,
        'efficiency_pu_div': (scr_gain_pu * mf / cost_pu) if cost_pu > 0 else 0,
        'scr_gain_pb_div': scr_gain_pb_div,
        'scr_net_pb_div': scr_net_pb_div,
        'scr_saving_div_usd': scr_saving_div,
        'notional_covered_pct': (current_nb * S_buy) / MV_eq * 100 if MV_eq > 0 else 0,
        'notional_capped': notional_capped,
        'nb_uncapped': nb_uncapped,
        'max_notional_pct': max_notional_pct,
        'dilution_ratio': mf,
        '_daily_coverage_dates': [c[0] for c in daily_cov],
        '_daily_coverage_values': [c[1] for c in daily_cov],
    }

    return TradeRecord(
        instrument_id=f'PUT_{moneyness:.0f}_{maturity_days}',
        instrument_type='vanilla_put',
        date_buy=date_buy, date_exp=date_exp,
        underlying_buy=S_buy, underlying_exp=S_exp, strike=K,
        nb_contracts=current_nb, premium_paid_usd=total_prem,
        budget_allocated_usd=total_prem,
        payoff_usd=payoff_tot, pnl_usd=pnl, pnl_pct_mv=pnl_pct,
        scr_before=scr_before, scr_after=scr_after,
        scr_saving_gross=scr_saving,
        net_capital_benefit=scr_saving - total_prem,
        M_net_pct=(scr_saving - total_prem) / MV_eq * 100 if MV_eq else 0,
        extras=extras)


# ═══════════════════════════════════════════════════════════════════════
# §7  AGGREGATION & DISTRIBUTION
# ═══════════════════════════════════════════════════════════════════════

def _mean(grp, col):
    if col in grp.columns: return grp[col].mean()
    return 0.0

def _parse_id(iid):
    parts = iid.split('_')
    return float(parts[1]), int(parts[2])

def aggregate_single_start(trade_dicts, start_id, start_date, end_date):
    df = pd.DataFrame(trade_dicts)
    if df.empty: return []
    window_days = (df['date_exp'].max() - df['date_buy'].min()).days
    window_years = window_days / 365.25 if window_days > 0 else 1.0
    records = []
    for iid, grp in df.groupby('instrument_id'):
        m, mt = _parse_id(iid)
        pnl_cum = grp['pnl_pct_mv'].sum()
        cum = grp['pnl_pct_mv'].cumsum()
        records.append({
            'start_id': start_id, 'start_date': start_date, 'end_date': end_date,
            'instrument_id': iid, 'moneyness': m, 'maturity': mt,
            'n_rolls': len(grp),
            'pnl_position_cum': pnl_cum,
            'pnl_position_ann': pnl_cum / window_years,
            'pnl_position_usd_ann': grp['pnl_usd'].sum() / window_years,
            'pnl_position_mean': grp['pnl_pct_mv'].mean(),
            'pnl_position_std': grp['pnl_pct_mv'].std(),
            'hit_rate': (grp['pnl_pct_mv'] > 0).mean() * 100,
            'max_drawdown': (cum - cum.cummax()).min(),
            'scr_gain_pu_iso': _mean(grp,'scr_gain_pu_iso'),
            'scr_gain_pu_div': _mean(grp,'scr_gain_pu_div'),
            'cost_pu': _mean(grp,'cost_pu'),
            'scr_net_pu_iso': _mean(grp,'scr_net_pu_iso'),
            'scr_net_pu_div': _mean(grp,'scr_net_pu_div'),
            'pnl_pu': _mean(grp,'pnl_pu'),
            'efficiency_pu_iso': _mean(grp,'efficiency_pu_iso'),
            'efficiency_pu_div': _mean(grp,'efficiency_pu_div'),
            'scr_gain_pb_div': _mean(grp,'scr_gain_pb_div'),
            'scr_net_pb_div': _mean(grp,'scr_net_pb_div'),
            'notional_covered_pct': _mean(grp,'notional_covered_pct'),
            'marginal_factor': _mean(grp,'marginal_factor'),
            'dilution_ratio': _mean(grp,'dilution_ratio'),
            'scr_nu_pct_avg': _mean(grp,'SCR_nu_pct'),
            'scr_hedged_pct_avg': _mean(grp,'SCR_hedged_pct'),
            'nb_contracts_avg': _mean(grp,'nb_contracts'),
            'n_adjustments_avg': _mean(grp,'n_adjustments'),
            'initial_premium_avg': _mean(grp,'initial_premium'),
            'adjustment_cost_avg': _mean(grp,'adjustment_cost'),
            'total_premium_avg': _mean(grp,'total_premium'),
            'scr_coverage_mean_avg': _mean(grp,'scr_coverage_mean'),
            'scr_coverage_min_avg': _mean(grp,'scr_coverage_min'),
            'scr_coverage_max_avg': _mean(grp,'scr_coverage_max'),
            'premium_pct_mv_avg': _mean(grp,'premium_pct_mv'),
            '_cum_pnl_dates': grp['date_exp'].values,
            '_cum_pnl_values': cum.values,
        })
    return records

def compute_distribution(df_agg, group_cols):
    agg_spec = {
        'n_starts': ('start_id', 'nunique'),
        'pnl_position_ann_mean': ('pnl_position_ann', 'mean'),
        'pnl_position_ann_median': ('pnl_position_ann', 'median'),
        'pnl_position_ann_p5': ('pnl_position_ann', lambda x: np.nanpercentile(x, 5)),
        'pnl_position_ann_p95': ('pnl_position_ann', lambda x: np.nanpercentile(x, 95)),
        'hit_rate_median': ('hit_rate', 'median'),
        'max_dd_mean': ('max_drawdown', 'mean'),
        'max_dd_worst': ('max_drawdown', 'min'),
        'max_dd_p5': ('max_drawdown', lambda x: np.nanpercentile(x, 5)),
    }
    for c in ['scr_gain_pu_iso','scr_gain_pu_div','cost_pu','scr_net_pu_iso',
              'scr_net_pu_div','pnl_pu','efficiency_pu_iso','efficiency_pu_div']:
        if c in df_agg.columns:
            agg_spec[f'{c}_median'] = (c, 'median')
            agg_spec[f'{c}_mean'] = (c, 'mean')
    for c in ['scr_gain_pb_div','scr_net_pb_div','notional_covered_pct']:
        if c in df_agg.columns:
            agg_spec[f'{c}_median'] = (c, 'median')
    for c in ['marginal_factor','dilution_ratio']:
        if c in df_agg.columns:
            agg_spec[f'{c}_avg'] = (c, 'mean')
    for c in ['n_adjustments_avg','scr_coverage_mean_avg','scr_coverage_min_avg',
              'scr_coverage_max_avg','premium_pct_mv_avg','total_premium_avg']:
        if c in df_agg.columns:
            agg_spec[f'{c}_median'] = (c, 'median')
            agg_spec[f'{c}_mean'] = (c, 'mean')
    return df_agg.groupby(group_cols).agg(**agg_spec)


# ═══════════════════════════════════════════════════════════════════════
# §8  REPORTING — 20-CHART PDF  (from plots_diversified.py)
# ═══════════════════════════════════════════════════════════════════════

import matplotlib; matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.cm as mcm
import matplotlib.dates as mdates
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.colors import LinearSegmentedColormap, Normalize
from matplotlib.lines import Line2D

# ─── Palette ───────────────────────────────────────────────────────────
C_ISO   = '#4393C3'; C_DIV = '#E6550D'; C_PREM = '#E74C3C'
C_NET   = '#1B9E77'; C_DIL = '#756BB1'; C_DD   = '#C0392B'
CMONEY  = {50:'#1B2A4A',60:'#D95F02',70:'#1B9E77',80:'#E7298A',90:'#7570B3',100:'#E6AB02'}
MMAT    = {30:'o',60:'s',90:'^',120:'D',150:'v',180:'p',360:'*',540:'h',720:'X'}
CBLUE   = {50:'#B3D9FF',60:'#80BFFF',70:'#4DA6FF',80:'#1A8CFF',90:'#0066CC',100:'#003D7A'}
CMAP_GB = LinearSegmentedColormap.from_list("GB",["#E0E0E0","#90CAF9","#1E88E5","#0D47A1"])
CMAP_BG = LinearSegmentedColormap.from_list("BG",["#0D47A1","#1E88E5","#90CAF9","#E0E0E0"])
CMAP_RG = LinearSegmentedColormap.from_list("RG",["#CC0000","#FF9999","#FFFFFF","#99FF99","#00CC00"])
CMAP_WR = LinearSegmentedColormap.from_list("WR",["#FFFFFF","#FFCCCC","#FF6666","#CC0000"])
COL_CI  = '#92C5DE'; COL_IQR = '#4393C3'; COL_MED = '#2166AC'

def _c(d, *cs):
    """Find first available column from candidates, with fallback substitutions."""
    cols = set(d.columns) if hasattr(d,'columns') else set()
    for c in cs:
        if c in cols: return c
    subs=[('_div_','_iso_'),('_div','_iso'),('_pb_','_pu_'),('_pb','_pu')]
    for c in cs:
        for o,n in subs:
            a=c.replace(o,n)
            if a in cols: return a
        a=c.replace('_div','_iso').replace('_pb','_pu')
        if a in cols: return a
    return cs[0]

def _heatmap(ax, pivot, title, cmap, fmt='.2f', vmin=None, vmax=None, label=''):
    """Draw annotated heatmap with smart text colour."""
    im = ax.imshow(pivot.values, cmap=cmap, aspect='auto', vmin=vmin, vmax=vmax)
    ax.set_title(title, fontsize=12, fontweight='bold', pad=15)
    ax.set_xticks(range(len(pivot.columns)))
    ax.set_xticklabels([f'{int(c)}j' for c in pivot.columns], fontsize=9)
    ax.set_yticks(range(len(pivot.index)))
    ax.set_yticklabels([f'K={int(m)}%' for m in pivot.index], fontsize=9)
    ax.set_xlabel('Maturity'); ax.set_ylabel('Moneyness')
    th = np.nanmean(pivot.values)
    for i in range(len(pivot.index)):
        for j in range(len(pivot.columns)):
            v = pivot.values[i,j]
            if not np.isnan(v):
                ax.text(j, i, format(v, fmt), ha='center', va='center',
                        fontsize=9, fontweight='bold',
                        color='white' if v > th else 'black')
    return im


# ── PART I: EFFICIENCY (Graphs 1–6) ──────────────────────────────────

def _g01_frontier_pb(dist, sd, ed):
    fig, ax = plt.subplots(figsize=(14, 9))
    xc = _c(dist,'scr_net_pb_div_median','scr_net_pb_div_mean')
    yc = _c(dist,'pnl_position_ann_median','pnl_position_ann_mean')
    for (m,t), r in dist.iterrows():
        c = CMONEY.get(int(m),'gray'); mk = MMAT.get(int(t),'o')
        if xc in r.index and yc in r.index:
            ax.scatter(r[xc], r[yc], color=c, marker=mk, s=120,
                       edgecolors='black', linewidth=0.5, alpha=0.9, zorder=5)
            ax.annotate(f'{int(t)}j', (r[xc], r[yc]), fontsize=6,
                        ha='center', va='bottom', textcoords='offset points', xytext=(0,6))
    ax.axhline(0, color='black', lw=0.8); ax.axvline(0, color='gray', lw=0.5, ls='--', alpha=0.5)
    for m, c in CMONEY.items():
        ax.scatter([], [], color=c, s=80, label=f'K={m}%', edgecolors='black')
    ax.legend(loc='upper left', fontsize=8, title='Moneyness', ncol=2)
    ax.set_xlabel('Net SCR Saving div per € spent (×)', fontsize=12)
    ax.set_ylabel('PnL annualized (% MV sleeve)', fontsize=12)
    ax.set_title(f'Graph 1 — Efficient Frontier per Budget (Diversified)\n{sd} to {ed}',
                 fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3); plt.tight_layout(); return fig

def _g02_frontier_pu(dist, sd, ed):
    fig, ax = plt.subplots(figsize=(14, 9))
    xc = _c(dist,'scr_net_pu_div_median','scr_net_pu_div_mean')
    yc = _c(dist,'pnl_pu_median','pnl_pu_mean')
    for (m,t), r in dist.iterrows():
        c = CMONEY.get(int(m),'gray'); mk = MMAT.get(int(t),'o')
        if xc in r.index and yc in r.index:
            ax.scatter(r[xc]*100, r[yc]*100, color=c, marker=mk, s=120,
                       edgecolors='black', linewidth=0.5, alpha=0.9, zorder=5)
            ax.annotate(f'{int(t)}j', (r[xc]*100, r[yc]*100), fontsize=6,
                        ha='center', va='bottom', textcoords='offset points', xytext=(0,6))
    ax.axhline(0, color='black', lw=0.8); ax.axvline(0, color='gray', lw=0.5, ls='--', alpha=0.5)
    for m, c in CMONEY.items():
        ax.scatter([], [], color=c, s=80, label=f'K={m}%', edgecolors='black')
    ax.legend(loc='upper left', fontsize=8, title='Moneyness', ncol=2)
    ax.set_xlabel('Net SCR Saving div (% spot per unit)', fontsize=12)
    ax.set_ylabel('PnL per unit (% spot)', fontsize=12)
    ax.set_title(f'Graph 2 — Efficient Frontier per Unit (Diversified)\n{sd} to {ed}',
                 fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3); plt.tight_layout(); return fig

def _g03_heatmaps_pb(dist):
    fig, (a1, a2) = plt.subplots(1, 2, figsize=(20, 8))
    sc = _c(dist,'scr_net_pb_div_median','scr_net_pb_div_mean')
    pc = _c(dist,'pnl_position_ann_median','pnl_position_ann_mean')
    im1 = _heatmap(a1, dist[sc].unstack(), 'Net SCR per Budget (div)\n(€ saved per € spent)', CMAP_GB)
    im2 = _heatmap(a2, dist[pc].unstack(), 'PnL Annualized (% MV sleeve)', CMAP_GB)
    fig.colorbar(im1, ax=a1, shrink=0.8); fig.colorbar(im2, ax=a2, shrink=0.8)
    fig.suptitle('Graph 3 — Per-Budget Heatmaps: SCR Saving & PnL',
                 fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout(); return fig

def _g04_heatmaps_pu(dist):
    fig, (a1, a2) = plt.subplots(1, 2, figsize=(20, 8))
    sc = _c(dist,'scr_net_pu_div_median','scr_net_pu_div_mean')
    pc = _c(dist,'pnl_pu_median','pnl_pu_mean')
    im1 = _heatmap(a1, (dist[sc]*100).unstack(), 'Net SCR per Unit (div)\n(% spot)', CMAP_GB)
    im2 = _heatmap(a2, (dist[pc]*100).unstack(), 'PnL per Unit (% spot)', CMAP_GB)
    fig.colorbar(im1, ax=a1, shrink=0.8); fig.colorbar(im2, ax=a2, shrink=0.8)
    fig.suptitle('Graph 4 — Per-Unit Heatmaps: SCR Saving & PnL',
                 fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout(); return fig

def _surface(dist, z_col, c_col, z_label, c_label, title, z_scale=1, c_scale=1):
    zc = _c(dist, z_col+'_median', z_col+'_mean')
    cc = _c(dist, c_col+'_median', c_col+'_mean')
    pz = dist[zc].unstack() * z_scale; pc = dist[cc].unstack() * c_scale
    X, Y = np.meshgrid(pz.columns.values, pz.index.values)
    Z = pz.values; C = pc.values
    cmap = LinearSegmentedColormap.from_list("VB",["#E0E0E0","#90CAF9","#1E88E5","#0000FF"])
    pp = C[C > 0]; vmax = np.percentile(pp, 20) if len(pp) > 0 else max(np.nanmax(C), 1)
    norm = Normalize(vmin=min(-1, np.nanmin(C)), vmax=vmax, clip=True)
    fig = plt.figure(figsize=(16, 11)); ax = fig.add_subplot(111, projection='3d')
    ax.plot_surface(X, Y, Z, facecolors=cmap(norm(C)), shade=True,
                    edgecolor='white', linewidth=0.03, alpha=0.95)
    mp = mcm.ScalarMappable(cmap=cmap, norm=norm); mp.set_array(C)
    fig.colorbar(mp, ax=ax, shrink=0.4, aspect=15, pad=0.1)
    ax.view_init(elev=25, azim=-45)
    ax.set_xlabel('\nMaturity (days)'); ax.set_ylabel('\nMoneyness (%)')
    ax.set_zlabel(z_label)
    ax.set_title(title, fontsize=15, fontweight='bold', pad=30)
    plt.tight_layout(); return fig

def _g05_surface_pb(dist):
    return _surface(dist, 'scr_net_pb_div', 'pnl_position_ann',
                    'Net SCR per € spent (×)', 'PnL (% MV)',
                    'Graph 5 — Per-Budget Efficiency Surface')

def _g06_surface_pu(dist):
    return _surface(dist, 'scr_net_pu_div', 'pnl_pu',
                    'Net SCR per unit (% spot)', 'PnL per unit (% spot)',
                    'Graph 6 — Per-Unit Efficiency Surface', z_scale=100, c_scale=100)


# ── PART II: SCR OVERVIEW (Graphs 7–14) ──────────────────────────────

def _g07_boxplots_scr_pb(df_agg, sd, ed, focus_moneys):
    if focus_moneys is None: focus_moneys = [80, 90, 100]
    col = _c(df_agg, 'scr_net_pb_div', 'scr_net_pb_iso')
    nrows = max(1, (len(focus_moneys) + 2) // 3)
    fig, axes = plt.subplots(nrows, 3, figsize=(20, 12))
    axes = np.atleast_2d(axes)
    fig.suptitle(f'Graph 7 — Net SCR Gain per Budget (Diversified)\n{sd} - {ed}',
                 fontsize=14, fontweight='bold')
    for idx, money in enumerate(focus_moneys):
        ax = axes[idx // 3, idx % 3]
        sub = df_agg[df_agg['moneyness'] == money]
        mats = sorted(sub['maturity'].unique())
        bd, bl = [], []
        for mat in mats:
            vals = sub[sub['maturity'] == mat][col].dropna().values
            if len(vals) > 0: bd.append(vals); bl.append(f'{int(mat)}j')
        if bd:
            bp = ax.boxplot(bd, labels=bl, patch_artist=True,
                            flierprops=dict(markersize=3, alpha=0.3))
            c = CMONEY.get(int(money), 'gray')
            for p in bp['boxes']: p.set_facecolor(c); p.set_alpha(0.5)
        ax.axhline(0, color='black', lw=0.8, ls='--')
        ax.set_title(f'Moneyness = {int(money)}%', fontweight='bold')
        ax.set_ylabel('Net SCR per € spent (×)'); ax.grid(axis='y', alpha=0.3)
    for idx in range(len(focus_moneys), nrows * 3):
        axes[idx // 3, idx % 3].set_visible(False)
    plt.tight_layout(); return fig

def _bars_scr_vs_prem(dist, gain_col, cost_col, eff_col, title, y_label):
    df = dist.reset_index().sort_values(['moneyness','maturity']).reset_index(drop=True)
    gc = _c(dist, gain_col+'_median', gain_col+'_mean')
    pc = _c(dist, cost_col+'_median', cost_col+'_mean')
    ec = _c(dist, eff_col+'_median', eff_col+'_mean')
    gross = df[gc].values if gc in df.columns else np.zeros(len(df))
    prem = df[pc].values if pc in df.columns else np.zeros(len(df))
    eff = df[ec].values if ec in df.columns else np.zeros(len(df))
    labels = [f"K={int(r['moneyness'])}%\n{int(r['maturity'])}j" for _, r in df.iterrows()]
    n = len(df); x = np.arange(n); w = 0.35
    bc = [CBLUE.get(int(r.get('moneyness', 90)), '#0066CC') for _, r in df.iterrows()]
    fig, ax1 = plt.subplots(figsize=(max(14, n * 0.6), 8))
    ax1.bar(x - w/2, gross, w, color=bc, edgecolor='white', alpha=0.9, label='SCR Gain (div)')
    ax1.bar(x + w/2, -np.abs(prem), w, color=C_PREM, edgecolor='white', alpha=0.7, label='Premium')
    ax1.set_ylabel(y_label, fontsize=11, fontweight='bold')
    ax1.axhline(0, color='black', lw=1); ax1.grid(axis='y', alpha=0.25, ls='--')
    ax2 = ax1.twinx()
    ax2.plot(x, eff, color=C_NET, lw=2.5, marker='D', markersize=6,
             markerfacecolor='white', markeredgecolor=C_NET, markeredgewidth=1.5,
             zorder=10, label='Efficiency')
    ax2.set_ylabel('Efficiency (×)', fontsize=11, fontweight='bold', color=C_NET)
    ax2.tick_params(axis='y', labelcolor=C_NET)
    ax1.set_xticks(x); ax1.set_xticklabels(labels, fontsize=8)
    if 'moneyness' in df.columns:
        pm = None
        for i, (_, r) in enumerate(df.iterrows()):
            m = r['moneyness']
            if pm is not None and m != pm:
                ax1.axvline(x=i - 0.5, color='gray', lw=0.8, ls=':', alpha=0.6)
            pm = m
    l1, la1 = ax1.get_legend_handles_labels()
    l2, la2 = ax2.get_legend_handles_labels()
    ax1.legend(l1 + l2, la1 + la2, loc='upper left', fontsize=9)
    ax1.set_title(title, fontsize=14, fontweight='bold', pad=15)
    plt.tight_layout(); return fig

def _g08_bars_pb(dist, sd, ed):
    return _bars_scr_vs_prem(dist, 'scr_gain_pb_div', 'scr_net_pb_div', 'efficiency_pu_div',
                             f'Graph 8 — SCR Gain vs Premium per Budget\n{sd} to {ed}',
                             '€ SCR per € spent')

def _g09_bars_pu(dist, sd, ed):
    df = dist.copy()
    for c in ['scr_gain_pu_div_median','scr_gain_pu_div_mean','cost_pu_median','cost_pu_mean']:
        if c in df.columns: df[c] = df[c] * 100
    return _bars_scr_vs_prem(df, 'scr_gain_pu_div', 'cost_pu', 'efficiency_pu_div',
                             f'Graph 9 — SCR Gain vs Premium per Unit (% spot)\n{sd} to {ed}',
                             '% of Spot per unit')

def _g10_coverage(dist, df_agg):
    fig, (a1, a2) = plt.subplots(1, 2, figsize=(20, 8))
    nc = _c(dist, 'notional_covered_pct_median', 'notional_covered_pct_mean')
    if nc in dist.columns:
        _heatmap(a1, dist[nc].unstack(), 'Notional Covered\n(% of sleeve MV)', CMAP_GB, fmt='.1f')
    if 'nb_contracts_avg' in df_agg.columns:
        nb_pivot = df_agg.groupby(['moneyness','maturity'])['nb_contracts_avg'].median().unstack()
    elif 'nb_contracts' in df_agg.columns:
        nb_pivot = df_agg.groupby(['moneyness','maturity'])['nb_contracts'].median().unstack()
    else:
        nb_pivot = None
    if nb_pivot is not None:
        _heatmap(a2, nb_pivot, 'Nb Puts Bought\n(median per roll)', CMAP_GB, fmt='.0f')
    else:
        a2.text(0.5, 0.5, 'nb_contracts not available', ha='center', va='center',
                transform=a2.transAxes)
    fig.suptitle('Graph 10 — Coverage: Notional & Number of Puts',
                 fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout(); return fig

def _g11_scr_nu_hedged(dist, df_agg):
    fig, (a1, a2) = plt.subplots(1, 2, figsize=(20, 8))
    nu_col = _c(df_agg, 'scr_nu_pct_avg', 'SCR_nu_pct')
    hd_col = _c(df_agg, 'scr_hedged_pct_avg', 'SCR_hedged_pct')
    pnu = df_agg.groupby(['moneyness','maturity'])[nu_col].median().unstack() \
          if nu_col in df_agg.columns else None
    phd = df_agg.groupby(['moneyness','maturity'])[hd_col].median().unstack() \
          if hd_col in df_agg.columns else None
    if pnu is not None and phd is not None:
        vmin = min(phd.min().min(), pnu.min().min())
        vmax = max(phd.max().max(), pnu.max().max())
        im1 = _heatmap(a1, pnu, 'SCR Before Hedges (% MV sleeve)\n(Unprotected)',
                        CMAP_BG, fmt='.1f', vmin=vmin, vmax=vmax)
        im2 = _heatmap(a2, phd, 'SCR After Hedges (% MV sleeve)\n(Protected)',
                        CMAP_BG, fmt='.1f', vmin=vmin, vmax=vmax)
        fig.colorbar(im1, ax=a1, shrink=0.8); fig.colorbar(im2, ax=a2, shrink=0.8)
    else:
        a1.text(0.5, 0.5, 'SCR data not available', ha='center', va='center',
                transform=a1.transAxes)
        a2.text(0.5, 0.5, 'SCR data not available', ha='center', va='center',
                transform=a2.transAxes)
    fig.suptitle('Graph 11 — SCR Before vs After Hedges (% MV sleeve)',
                 fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout(); return fig

def _g12_waterfall(dist):
    df = dist.reset_index()
    gic = _c(dist, 'scr_gain_pu_iso_median', 'scr_gain_pu_iso_mean')
    gdc = _c(dist, 'scr_gain_pu_div_median', 'scr_gain_pu_div_mean')
    cc  = _c(dist, 'cost_pu_median', 'cost_pu_mean')
    ndc = _c(dist, 'scr_net_pu_div_median', 'scr_net_pu_div_mean')
    targets = []
    for money in sorted(df['moneyness'].unique()):
        sub = df[df['moneyness'] == money]
        if 180 in sub['maturity'].values:
            targets.append(sub[sub['maturity'] == 180].iloc[0])
        elif len(sub) > 0:
            targets.append(sub.iloc[len(sub) // 2])
    if not targets: return plt.figure()
    n = len(targets)
    fig, axes = plt.subplots(1, n, figsize=(6 * n, 8), sharey=True)
    if n == 1: axes = [axes]
    for idx, row in enumerate(targets):
        ax = axes[idx]; money = int(row['moneyness']); mat = int(row['maturity'])
        gi = row.get(gic, 0) * 100; gd = row.get(gdc, 0) * 100
        co = row.get(cc, 0) * 100;  nd = row.get(ndc, 0) * 100
        dil = gi - gd
        bars = [gi, -dil, -co, nd]
        lbls = ['Gross SCR\n(Isolated)', 'Dilution\nLoss', 'Premium\nCost', 'Net SCR\n(Diversified)']
        colors = ['#2166AC', C_DIL, C_PREM, C_NET]
        bots = [0, gi, gd, 0]
        for i, (v, cl, b) in enumerate(zip(bars, colors, bots)):
            if i < 3:
                ax.bar(i, v, bottom=b, color=cl, width=0.6, edgecolor='white', alpha=0.85)
                ax.text(i, b + v/2, f'{v:+.2f}%', ha='center', va='center', fontsize=9,
                        fontweight='bold', color='white' if abs(v) > 0.3 else 'black')
            else:
                ax.bar(i, v, color=cl, width=0.6, edgecolor='white', alpha=0.85)
                ax.text(i, v + 0.05, f'{v:.2f}%', ha='center', va='bottom', fontsize=10,
                        fontweight='bold', color=cl)
        ax.set_xticks(range(4)); ax.set_xticklabels(lbls, fontsize=9)
        ax.axhline(0, color='black', lw=0.8)
        ax.set_title(f'K={money}% T={mat}j', fontweight='bold')
        ax.grid(axis='y', alpha=0.3)
        if idx == 0: ax.set_ylabel('% of Spot (per unit)', fontsize=11)
    fig.suptitle('Graph 12 — Per-Unit Waterfall: Gross → Dilution → Premium → Net',
                 fontsize=15, fontweight='bold', y=1.02)
    plt.tight_layout(); return fig

def _g13_marginal_factor(dist):
    fc = 'marginal_factor_avg'
    if fc not in dist.columns: return None
    pivot = dist[fc].unstack()
    fig, ax = plt.subplots(figsize=(12, 8))
    im = ax.imshow(pivot.values, cmap=CMAP_RG, aspect='auto', vmin=0.5, vmax=1.0)
    ax.set_title('Graph 13 — Marginal Diversification Factor\n(1.0 = no dilution)',
                 fontsize=14, fontweight='bold', pad=15)
    ax.set_xticks(range(len(pivot.columns)))
    ax.set_xticklabels([f'{int(c)}j' for c in pivot.columns], fontsize=10)
    ax.set_yticks(range(len(pivot.index)))
    ax.set_yticklabels([f'K={int(m)}%' for m in pivot.index], fontsize=10)
    ax.set_xlabel('Maturity'); ax.set_ylabel('Moneyness')
    for i in range(len(pivot.index)):
        for j in range(len(pivot.columns)):
            v = pivot.values[i, j]
            if not np.isnan(v):
                ax.text(j, i, f'{v:.3f}', ha='center', va='center', fontsize=10,
                        fontweight='bold', color='black' if v > 0.7 else 'white')
    fig.colorbar(im, ax=ax, label='Marginal Factor', shrink=0.8)
    plt.tight_layout(); return fig

def _g14_sa(sa_series):
    fig, ax = plt.subplots(figsize=(14, 6))
    shock = 0.39 + sa_series
    ax.plot(shock.index, shock.values * 100, color='darkred', lw=1.8,
            label='Equity Shock (39%+SA)')
    ax.axhline(39, color='black', lw=1.2, alpha=0.6, label='Base (39%)')
    ax.axhline(49, color='#C0392B', lw=1, ls='--', label='Cap (49%)')
    ax.axhline(29, color='#27AE60', lw=1, ls='--', label='Floor (29%)')
    ax.fill_between(shock.index, 39, shock.values * 100,
                    where=(shock.values * 100 >= 39), color='#C0392B', alpha=0.15)
    ax.fill_between(shock.index, 39, shock.values * 100,
                    where=(shock.values * 100 < 39), color='#27AE60', alpha=0.15)
    ax.set_ylabel('Capital Charge (%)', fontsize=11, fontweight='bold')
    ax.set_title('Graph 14 — Solvency II Symmetric Adjustment',
                 fontsize=14, fontweight='bold', pad=15)
    ax.set_ylim(25, 53); ax.grid(True, ls=':', alpha=0.5)
    ax.legend(loc='upper left', fontsize=10, ncol=2)
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
    plt.tight_layout(); return fig


# ── PART III: PnL (Graphs 15–18) ─────────────────────────────────────

def _g15_pnl_fans(df_agg, cum_pnl_series, spot_series, candidate_starts,
                   window_days, n_starts, focus_moneys, focus_mats, window_years):
    if focus_moneys is None: focus_moneys = [80, 90, 100]
    if focus_mats is None: focus_mats = [30, 90, 180, 360, 540]
    peak = spot_series.expanding(min_periods=1).max()
    dd = (spot_series / peak) - 1
    nrows = len(focus_mats); ncols = len(focus_moneys)
    fig, axes = plt.subplots(nrows, ncols, figsize=(28, 30), sharex=True)
    if window_years is None: window_years = window_days / 365.25
    fig.suptitle(f'Graph 15 — Cumulative PnL with CI ({n_starts} entries) vs Drawdowns\n'
                 f'Window = {window_years:.1f}y', fontsize=18, fontweight='bold')
    for ci, mat in enumerate(focus_mats):
        for ri, money in enumerate(focus_moneys):
            ax = axes[ci, ri] if nrows > 1 else axes[ri]
            ax_dd = ax.twinx()
            s0 = candidate_starts[0]
            el = candidate_starts[-1] + pd.Timedelta(days=window_days)
            ddp = dd.loc[s0:el]
            ax_dd.fill_between(ddp.index, ddp.values * 100, 0, color=C_DD, alpha=0.06)
            ax_dd.set_ylim(-60, 5)
            paths = []
            for sid in range(n_starts):
                key = (money, mat, sid)
                if key in cum_pnl_series: paths.append(cum_pnl_series[key])
            if not paths:
                ax.set_title(f'K={int(money)}% T={int(mat)}j — No data', fontsize=9)
                continue
            ads = set()
            for d, _ in paths: ads.update(d)
            cd = np.array(sorted(ads))
            matrix = np.full((len(paths), len(cd)), np.nan)
            for i, (d, c) in enumerate(paths):
                ps = pd.Series(c, index=d)
                matrix[i, :] = ps.reindex(cd, method='ffill').values
            p5  = np.nanpercentile(matrix, 5, axis=0)
            p25 = np.nanpercentile(matrix, 25, axis=0)
            p50 = np.nanpercentile(matrix, 50, axis=0)
            p75 = np.nanpercentile(matrix, 75, axis=0)
            p95 = np.nanpercentile(matrix, 95, axis=0)
            ax.fill_between(cd, p5, p95, alpha=0.12, color=COL_CI)
            ax.fill_between(cd, p25, p75, alpha=0.25, color=COL_IQR)
            ax.plot(cd, p50, color=COL_MED, lw=1.8)
            ax.axhline(0, color='black', lw=0.6, alpha=0.5)
            ax.set_title(f'K={int(money)}%  T={int(mat)}j', fontsize=10, fontweight='bold')
            ax.grid(True, alpha=0.15)
            ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
    plt.tight_layout(rect=[0, 0.02, 1, 0.96]); return fig

def _g16_boxplots_pnl(df_agg, sd, ed, focus_moneys):
    if focus_moneys is None: focus_moneys = [80, 90, 100]
    col = _c(df_agg, 'pnl_position_ann', 'pnl_annualized_pct')
    nrows = max(1, (len(focus_moneys) + 2) // 3)
    fig, axes = plt.subplots(nrows, 3, figsize=(20, 12))
    axes = np.atleast_2d(axes)
    fig.suptitle(f'Graph 16 — PnL Distribution (annualised, % MV sleeve)\n{sd} - {ed}',
                 fontsize=14, fontweight='bold')
    for idx, money in enumerate(focus_moneys):
        ax = axes[idx // 3, idx % 3]
        sub = df_agg[df_agg['moneyness'] == money]
        mats = sorted(sub['maturity'].unique())
        bd, bl = [], []
        for mat in mats:
            vals = sub[sub['maturity'] == mat][col].dropna().values
            if len(vals) > 0: bd.append(vals); bl.append(f'{int(mat)}j')
        if bd:
            bp = ax.boxplot(bd, labels=bl, patch_artist=True,
                            flierprops=dict(markersize=3, alpha=0.3))
            c = CMONEY.get(int(money), 'gray')
            for p in bp['boxes']: p.set_facecolor(c); p.set_alpha(0.5)
        ax.axhline(0, color='black', lw=0.8)
        ax.set_title(f'Moneyness = {int(money)}%', fontweight='bold')
        ax.set_ylabel('PnL (annualised %)'); ax.grid(axis='y', alpha=0.3)
    for idx in range(len(focus_moneys), nrows * 3):
        axes[idx // 3, idx % 3].set_visible(False)
    plt.tight_layout(); return fig

def _g17_iv_premium_nb(df_detail, focus_moneys, focus_mats):
    if focus_moneys is None: focus_moneys = [80, 90, 100]
    if focus_mats is None: focus_mats = [30, 90, 180, 360, 540]
    ncols = len(focus_moneys); nrows = len(focus_mats)
    fig, axes = plt.subplots(nrows * 3, ncols, figsize=(28, nrows * 9),
                             sharex=True,
                             gridspec_kw={'height_ratios': [2, 2, 1] * nrows})
    fig.suptitle('Graph 17 — Implied Volatility, Put Premium & Number of Puts Over Time',
                 fontsize=18, fontweight='bold', y=0.995)
    for ci, mat in enumerate(focus_mats):
        for ri, money in enumerate(focus_moneys):
            ax_iv = axes[ci * 3, ri]
            ax_pr = axes[ci * 3 + 1, ri]
            ax_nb = axes[ci * 3 + 2, ri]
            sub = df_detail[
                (df_detail['moneyness'] == money) & (df_detail['maturity'] == mat)
            ]
            if sub.empty:
                for a in [ax_iv, ax_pr, ax_nb]:
                    a.text(0.5, 0.5, 'No data', ha='center', va='center',
                           transform=a.transAxes, fontsize=9, color='gray')
                ax_iv.set_title(f'K={int(money)}%  T={int(mat)}j', fontsize=10,
                                fontweight='bold')
                continue
            sub = sub.sort_values('date_buy')
            dates = sub['date_buy'].values
            iv = sub['iv_buy'].values * 100 if 'iv_buy' in sub.columns else np.zeros(len(sub))
            prem = sub['put_price_pct_spot'].values if 'put_price_pct_spot' in sub.columns \
                   else np.zeros(len(sub))
            nb = sub['nb_contracts'].values if 'nb_contracts' in sub.columns \
                 else np.zeros(len(sub))
            # Row 1: IV
            ax_iv.plot(dates, iv, color='#1B2A4A', lw=1.3, alpha=0.9)
            ax_iv.fill_between(dates, iv, alpha=0.08, color='#1B2A4A')
            ax_iv.set_ylabel('IV (%)', fontsize=8, color='#1B2A4A')
            ax_iv.tick_params(axis='y', labelcolor='#1B2A4A', labelsize=7)
            ax_iv.grid(True, alpha=0.15, ls='--')
            ax_iv.set_title(f'K={int(money)}%  T={int(mat)}j', fontsize=10, fontweight='bold')
            # Row 2: Premium
            ax_pr.plot(dates, prem, color=C_PREM, lw=1.3, alpha=0.9)
            ax_pr.fill_between(dates, prem, alpha=0.1, color=C_PREM)
            ax_pr.set_ylabel('Prem (% spot)', fontsize=8, color=C_PREM)
            ax_pr.tick_params(axis='y', labelcolor=C_PREM, labelsize=7)
            ax_pr.grid(True, alpha=0.15, ls='--')
            # Row 3: Nb puts
            ax_nb.step(dates, nb, where='mid', color=C_NET, lw=1.0, alpha=0.8)
            ax_nb.fill_between(dates, nb, step='mid', alpha=0.15, color=C_NET)
            ax_nb.set_ylabel('Nb puts', fontsize=8, color=C_NET)
            ax_nb.tick_params(axis='y', labelcolor=C_NET, labelsize=7)
            ax_nb.grid(True, alpha=0.15, ls='--')
            ax_nb.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
            if ci == 0 and ri == 0:
                handles = [
                    Line2D([0],[0], color='#1B2A4A', lw=1.5, label='Implied Volatility (%)'),
                    Line2D([0],[0], color=C_PREM, lw=1.5, label='Put Premium (% spot)'),
                    Line2D([0],[0], color=C_NET, lw=1.5, label='Nb Puts Bought'),
                ]
                ax_iv.legend(handles=handles, fontsize=7, loc='upper right',
                             ncol=3, frameon=True, fancybox=True)
    plt.tight_layout(rect=[0, 0.01, 1, 0.985]); return fig

def _g18_drawdowns(spot_series):
    peak = spot_series.expanding(min_periods=1).max()
    dd = (spot_series / peak) - 1
    is_dd = dd <= -0.10; diff = is_dd.astype(int).diff()
    dd_starts = diff[diff == 1].index; dd_ends = diff[diff == -1].index
    if is_dd.iloc[-1]:
        dd_ends = dd_ends.append(pd.Index([is_dd.index[-1]]))
    fig, ax1 = plt.subplots(figsize=(14, 7))
    ax1.plot(spot_series.index, spot_series.values, color='#1B2A4A', lw=1.5,
             label='S&P 500 Spot')
    ax1.set_ylabel('Spot ($)', color='#1B2A4A', fontweight='bold')
    ax1.grid(True, alpha=0.2, ls='--')
    ax2 = ax1.twinx()
    ax2.plot(dd.index, dd.values * 100, color=C_DD, lw=1, alpha=0.4)
    ax2.fill_between(dd.index, dd.values * 100, 0, where=(dd <= 0), color=C_DD, alpha=0.1)
    ax2.set_ylabel('Drawdown (%)', color=C_DD, fontweight='bold')
    ax2.set_ylim(-60, 5)
    for i, (s, e) in enumerate(zip(dd_starts, dd_ends)):
        ax1.axvspan(s, e, color='gray', alpha=0.15,
                    label='Stress Zone' if i == 0 else '')
    plt.title('Graph 18 — Market Regime: S&P 500 Spot and Drawdowns (10%)',
              fontsize=14, fontweight='bold', pad=15)
    ax1.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
    l1, la1 = ax1.get_legend_handles_labels()
    l2, la2 = ax2.get_legend_handles_labels()
    ax1.legend(l1 + l2, la1 + la2, loc='upper left', fontsize=10)
    plt.tight_layout(); return fig


def _g19_coverage_through_time(df_detail, focus_moneys, focus_mats,
                                target, band_lo, band_hi):
    """Graph 19 — SCR Coverage Ratio through time (fan chart per instrument).
    Extracts the daily coverage time series stored in each trade record,
    stitches consecutive rolls, then builds a fan chart across backtest starts.
    """
    if focus_moneys is None: focus_moneys = [90, 100]
    if focus_mats is None: focus_mats = [30, 90, 180, 360, 540]
    if df_detail.empty or '_daily_coverage_dates' not in df_detail.columns:
        return None

    nrows = len(focus_mats); ncols = len(focus_moneys)
    fig, axes = plt.subplots(nrows, ncols, figsize=(8 * ncols, 5 * nrows), sharex=True)
    if nrows == 1 and ncols == 1: axes = np.array([[axes]])
    elif nrows == 1: axes = axes[np.newaxis, :]
    elif ncols == 1: axes = axes[:, np.newaxis]

    fig.suptitle(
        f'Graph 19 — SCR Coverage Ratio Through Time\n'
        f'Target = {target:.0%}  Band = [{band_lo:.0%} – {band_hi:.0%}]',
        fontsize=16, fontweight='bold')

    for ri, mat in enumerate(focus_mats):
        for ci, money in enumerate(focus_moneys):
            ax = axes[ri, ci]
            sub = df_detail[
                (df_detail['moneyness'] == money) & (df_detail['maturity'] == mat)
            ]
            if sub.empty:
                ax.text(0.5, 0.5, 'No data', ha='center', va='center',
                        transform=ax.transAxes, fontsize=9, color='gray')
                ax.set_title(f'K={int(money)}%  T={int(mat)}j', fontsize=10,
                             fontweight='bold')
                continue

            # Build one stitched coverage path per start_id
            paths = {}  # start_id -> (dates_array, values_array)
            for _, row in sub.iterrows():
                sid = row.get('start_id', 0)
                d_list = row['_daily_coverage_dates']
                v_list = row['_daily_coverage_values']
                if d_list is None or v_list is None or len(d_list) == 0:
                    continue
                if sid not in paths:
                    paths[sid] = (list(d_list), list(v_list))
                else:
                    paths[sid] = (paths[sid][0] + list(d_list),
                                  paths[sid][1] + list(v_list))

            if not paths:
                ax.text(0.5, 0.5, 'No coverage data', ha='center', va='center',
                        transform=ax.transAxes, fontsize=9, color='gray')
                ax.set_title(f'K={int(money)}%  T={int(mat)}j', fontsize=10,
                             fontweight='bold')
                continue

            # Collect all dates to build a common grid
            all_dates_set = set()
            for d_list, _ in paths.values():
                all_dates_set.update(d_list)
            common_dates = np.array(sorted(all_dates_set))

            # Build matrix: n_paths × n_dates
            matrix = np.full((len(paths), len(common_dates)), np.nan)
            for i, (sid, (d_list, v_list)) in enumerate(paths.items()):
                ps = pd.Series(v_list, index=pd.DatetimeIndex(d_list))
                ps = ps[~ps.index.duplicated(keep='last')].sort_index()
                matrix[i, :] = ps.reindex(common_dates, method='ffill').values

            # Percentile bands
            p5  = np.nanpercentile(matrix, 5,  axis=0) * 100
            p25 = np.nanpercentile(matrix, 25, axis=0) * 100
            p50 = np.nanpercentile(matrix, 50, axis=0) * 100
            p75 = np.nanpercentile(matrix, 75, axis=0) * 100
            p95 = np.nanpercentile(matrix, 95, axis=0) * 100

            ax.fill_between(common_dates, p5, p95, alpha=0.10, color=COL_CI,
                            label='5–95 pctl')
            ax.fill_between(common_dates, p25, p75, alpha=0.25, color=COL_IQR,
                            label='25–75 pctl')
            ax.plot(common_dates, p50, color=COL_MED, lw=1.6, label='Median')

            # Target + bands
            ax.axhline(target * 100, color='#2166AC', lw=1.2, ls='-', alpha=0.7,
                       label=f'Target ({target:.0%})')
            ax.axhline(band_lo * 100, color='#E74C3C', lw=0.9, ls='--', alpha=0.5,
                       label=f'Band [{band_lo:.0%}–{band_hi:.0%}]')
            ax.axhline(band_hi * 100, color='#E74C3C', lw=0.9, ls='--', alpha=0.5)

            ax.set_title(f'K={int(money)}%  T={int(mat)}j', fontsize=10, fontweight='bold')
            ax.set_ylabel('Coverage (%)', fontsize=9)
            ax.grid(True, alpha=0.15, ls='--')
            ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
            ax.set_ylim(max(0, (band_lo - 0.15) * 100), min(100, (band_hi + 0.20) * 100))

            if ri == 0 and ci == 0:
                ax.legend(fontsize=7, loc='upper right', ncol=3, frameon=True, fancybox=True)

    plt.tight_layout(rect=[0, 0.01, 1, 0.97])
    return fig


def _g20_iv_skew_through_time(df_raw, focus_mats):
    """Graph 20 — Options Skew Through Time.
    For each focus maturity, plots the IV for all available moneyness levels
    through time, plus a skew metric (IV_OTM - IV_ATM).
    """
    if focus_mats is None: focus_mats = [30, 90, 180, 360, 540]
    if df_raw is None or df_raw.empty:
        return None

    moneyness_levels = sorted(df_raw['moneyness'].unique())

    # Colour palette: deep blue for low moneyness (deep OTM) → amber for ATM
    n_levels = len(moneyness_levels)
    cmap_skew = plt.cm.coolwarm
    m_colors = {m: cmap_skew(i / max(1, n_levels - 1))
                for i, m in enumerate(moneyness_levels)}

    nrows = len(focus_mats)
    fig, axes = plt.subplots(nrows, 2, figsize=(24, 5 * nrows),
                             gridspec_kw={'width_ratios': [3, 2]})
    if nrows == 1: axes = axes[np.newaxis, :]

    fig.suptitle('Graph 20 — Implied Volatility Skew Through Time',
                 fontsize=16, fontweight='bold')

    for ri, mat in enumerate(focus_mats):
        ax_iv = axes[ri, 0]
        ax_sk = axes[ri, 1]

        sub = df_raw[df_raw['maturity'] == mat].copy()
        if sub.empty:
            for a in [ax_iv, ax_sk]:
                a.text(0.5, 0.5, 'No data', ha='center', va='center',
                       transform=a.transAxes, fontsize=9, color='gray')
            ax_iv.set_title(f'T = {int(mat)}j — IV Smile', fontsize=10, fontweight='bold')
            ax_sk.set_title(f'T = {int(mat)}j — Skew', fontsize=10, fontweight='bold')
            continue

        # Left panel: IV lines per moneyness
        for m in moneyness_levels:
            ms = sub[sub['moneyness'] == m].sort_values('Date')
            if ms.empty: continue
            iv_vals = ms['iv'].values * 100
            dates_m = ms['Date'].values
            lw = 1.8 if m == 100 else 1.0
            alpha = 1.0 if m in (80, 90, 100) else 0.5
            ax_iv.plot(dates_m, iv_vals, color=m_colors[m], lw=lw, alpha=alpha,
                       label=f'K={int(m)}%')

        ax_iv.set_title(f'T = {int(mat)}j — IV by Moneyness', fontsize=11, fontweight='bold')
        ax_iv.set_ylabel('Implied Volatility (%)', fontsize=9)
        ax_iv.grid(True, alpha=0.15, ls='--')
        ax_iv.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
        ax_iv.legend(fontsize=7, loc='upper left', ncol=min(5, n_levels), frameon=True)

        # Right panel: Skew = IV(OTM) - IV(ATM) for selected pairs
        # Pivot to get daily IV per moneyness
        piv = sub.pivot_table(index='Date', columns='moneyness', values='iv',
                              aggfunc='first').sort_index() * 100
        if 100 in piv.columns:
            atm = piv[100]
            skew_pairs = [(m, f'K={int(m)}% – K=100%') for m in [80, 90]
                          if m in piv.columns and m in moneyness_levels]
            for m, label in skew_pairs:
                skew_val = piv[m] - atm
                c = m_colors.get(m, 'gray')
                ax_sk.plot(piv.index, skew_val, color=c, lw=1.4, alpha=0.9, label=label)
                ax_sk.fill_between(piv.index, skew_val, 0, color=c, alpha=0.06)
            ax_sk.axhline(0, color='black', lw=0.8, alpha=0.5)
            ax_sk.set_title(f'T = {int(mat)}j — Skew (vs ATM)', fontsize=11, fontweight='bold')
            ax_sk.set_ylabel('IV Spread (pp)', fontsize=9)
            ax_sk.grid(True, alpha=0.15, ls='--')
            ax_sk.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
            ax_sk.legend(fontsize=8, loc='upper left', frameon=True)
        else:
            ax_sk.text(0.5, 0.5, 'No ATM (K=100%) data', ha='center', va='center',
                       transform=ax_sk.transAxes, fontsize=9, color='gray')
            ax_sk.set_title(f'T = {int(mat)}j — Skew', fontsize=10, fontweight='bold')

    plt.tight_layout(rect=[0, 0.01, 1, 0.97])
    return fig


# ── ORCHESTRATOR ──────────────────────────────────────────────────────

def generate_report(
    dist, df_agg, df_detail,
    spot_series, sa_series,
    cum_pnl_series, candidate_starts,
    start_date, end_date,
    window_days, n_starts,
    window_years=None,
    focus_moneys=None, focus_mats=None,
    folder=None,
    df_raw=None,
    target_coverage=0.30, band_low=0.25, band_high=0.35,
):
    """Generate ONE unified PDF report with 20 charts."""
    if folder:
        os.makedirs(folder, exist_ok=True)
    if window_years is None: window_years = window_days / 365.25
    if focus_moneys is None: focus_moneys = [80, 90, 100]
    if focus_mats is None: focus_mats = [30, 90, 180, 360, 540]
    sd, ed = str(start_date), str(end_date)

    figs = {}

    # Part I — Efficiency
    figs['g01'] = _g01_frontier_pb(dist, sd, ed)
    figs['g02'] = _g02_frontier_pu(dist, sd, ed)
    figs['g03'] = _g03_heatmaps_pb(dist)
    figs['g04'] = _g04_heatmaps_pu(dist)
    figs['g05'] = _g05_surface_pb(dist)
    figs['g06'] = _g06_surface_pu(dist)

    # Part II — SCR Overview
    figs['g07'] = _g07_boxplots_scr_pb(df_agg, sd, ed, focus_moneys)
    figs['g08'] = _g08_bars_pb(dist, sd, ed)
    figs['g09'] = _g09_bars_pu(dist, sd, ed)
    figs['g10'] = _g10_coverage(dist, df_agg)
    figs['g11'] = _g11_scr_nu_hedged(dist, df_agg)
    figs['g12'] = _g12_waterfall(dist)
    figs['g13'] = _g13_marginal_factor(dist)
    figs['g14'] = _g14_sa(sa_series)

    # Part III — PnL
    figs['g15'] = _g15_pnl_fans(df_agg, cum_pnl_series, spot_series,
                                 candidate_starts, window_days, n_starts,
                                 focus_moneys, focus_mats, window_years)
    figs['g16'] = _g16_boxplots_pnl(df_agg, sd, ed, focus_moneys)
    figs['g17'] = _g17_iv_premium_nb(df_detail, focus_moneys, focus_mats)
    figs['g18'] = _g18_drawdowns(spot_series)

    # Part IV — Coverage & Skew
    figs['g19'] = _g19_coverage_through_time(df_detail, focus_moneys, focus_mats,
                                              target_coverage, band_low, band_high)
    figs['g20'] = _g20_iv_skew_through_time(df_raw, focus_mats)

    if folder:
        pdf_path = os.path.join(folder, 'Rapport_Hedging.pdf')
        with PdfPages(pdf_path) as pdf:
            for name in sorted(figs.keys()):
                fig = figs[name]
                if fig is not None:
                    pdf.savefig(fig, bbox_inches='tight')
            d = pdf.infodict()
            d['Title'] = f'Hedging Report — {sd} to {ed}'
        n_saved = sum(1 for f in figs.values() if f is not None)
        print(f"Report saved: {pdf_path} ({n_saved} charts)")

    plt.close('all')
    return figs


# ═══════════════════════════════════════════════════════════════════════
# §9  MAIN PIPELINE
# ═══════════════════════════════════════════════════════════════════════

def main():
    print("=" * 70)
    print(f"SCR-TARGET BACKTEST — Coverage {TARGET_COVERAGE:.0%} [{BAND_LOW:.0%}–{BAND_HIGH:.0%}]")
    print(f"Roll mode: {ROLL_MODE}")
    if MAX_NOTIONAL_PCT is not None:
        print(f"Notional cap: {MAX_NOTIONAL_PCT:.0%} of equity sleeve")
    else:
        print(f"Notional cap: disabled (uncapped)")
    print("=" * 70)
    t_total = time.time()

    # ── 1. Load data ──────────────────────────────────────────────
    df_raw, spot_series, sa_series, all_dates = load_data(DATA_PATH)

    # ── 2. Build SCR aggregator ───────────────────────────────────
    agg = SCRAggregator(scenario=RATE_SCENARIO)
    equity_mv = MV_EQUITY_T0
    sa_t0 = sa_series.iloc[0] if len(sa_series) > 0 else 0.0
    agg.set_scr('equity', equity_mv * (0.39 + sa_t0))
    for mod, val in SCR_OTHER_MODULES.items():
        agg.set_scr(mod, val)
    print(f"\n{agg.summary()}")

    # ── 3. Instrument grid ────────────────────────────────────────
    combos = df_raw.groupby(['moneyness','maturity']).size().reset_index(name='n')
    if MONEYNESS_FILTER:
        combos = combos[combos['moneyness'].isin(MONEYNESS_FILTER)]
    if MATURITY_FILTER:
        combos = combos[combos['maturity'].isin(MATURITY_FILTER)]
    instruments = [(r['moneyness'], int(r['maturity'])) for _, r in combos.iterrows()]
    print(f"\nInstruments: {len(instruments)}")

    # ── 4. Roll scheduler ────────────────────────────────────────
    signal_series = None
    if ROLL_MODE == 'signal' and SIGNAL_COLUMN in df_raw.columns:
        sig_df = df_raw[['Date', SIGNAL_COLUMN]].drop_duplicates('Date').set_index('Date')
        signal_series = sig_df[SIGNAL_COLUMN]
        print(f"Signal column '{SIGNAL_COLUMN}' loaded: {len(signal_series)} values")

    def make_scheduler(maturity_days):
        if ROLL_MODE == 'signal' and signal_series is not None:
            return SignalDrivenRoll(signal_series, SIGNAL_THRESHOLD,
                                   SIGNAL_MODE, SIGNAL_HOLD_DAYS, SIGNAL_COOLDOWN)
        return SystematicRoll()

    # ── 5. Window & starts ────────────────────────────────────────
    start_dt, end_dt = pd.Timestamp(START_DATE), pd.Timestamp(END_DATE)
    dates_in_range = [d for d in all_dates if start_dt <= d <= end_dt]
    n_td = len(dates_in_range)
    window_days = int((n_td - N_STARTS) / 252 * 365.25)
    window_years = window_days / 365.25
    candidate_starts = dates_in_range[:N_STARTS]
    last_end = candidate_starts[-1] + pd.Timedelta(days=window_days)
    if last_end > end_dt:
        window_days = (end_dt - candidate_starts[-1]).days
        window_years = window_days / 365.25
    n_starts = len(candidate_starts)
    spot_t0 = spot_series.loc[dates_in_range[0]]

    print(f"Window: {window_years:.2f}y | Starts: {n_starts}")

    # ── FIX #9: Build instrument-specific daily lookup ────────────
    print("Building instrument-specific daily lookup...")
    df_raw_indexed = df_raw.set_index('Date').sort_index()
    instr_daily = {}
    for money, mat in instruments:
        mask = (df_raw_indexed['moneyness'] == money) & (df_raw_indexed['maturity'] == mat)
        sub = df_raw_indexed[mask]
        sub = sub[~sub.index.duplicated(keep='first')]
        instr_daily[(money, mat)] = sub
    print(f"  Built lookup for {len(instr_daily)} instruments")

    # ── 6. Main loop ──────────────────────────────────────────────
    print(f"\n{'='*70}\nRUNNING BACKTESTS\n{'='*70}")
    capital = SolvencyIIEquity()
    all_detail, all_agg, cum_pnl = [], [], {}
    t0 = time.time()

    for sid, sd in enumerate(candidate_starts):
        ed = sd + pd.Timedelta(days=window_days)
        spot_ref = spot_series.loc[sd] if sd in spot_series.index else spot_t0

        if sd in sa_series.index:
            sa_cur = sa_series.loc[sd]
            mv_ratio = spot_ref / spot_t0
            agg.set_scr('equity', equity_mv * mv_ratio * (0.39 + sa_cur))

        # FIX #2: Snapshot before instrument loop
        agg_snapshot = agg.snapshot()

        mask = (df_raw['Date'] >= sd) & (df_raw['Date'] <= ed)
        df_w = df_raw[mask]
        if df_w.empty: continue

        trades_this = []
        for money, mat in instruments:
            agg.restore(agg_snapshot)  # FIX #2

            sub = df_w[(df_w['moneyness']==money) & (df_w['maturity']==mat)] \
                  .sort_values('Date').reset_index(drop=True)
            if sub.empty: continue

            scheduler = make_scheduler(mat)
            schedule = scheduler.generate_schedule(sub['Date'].values, mat)
            if not schedule: continue

            # FIX #9: instrument-specific daily data
            df_daily_instr = instr_daily.get((money, mat))
            if df_daily_instr is None: continue
            df_daily_w = df_daily_instr[(df_daily_instr.index >= sd) &
                                        (df_daily_instr.index <= ed)]

            trades = run_scr_target_roll(
                sub, df_daily_w, schedule, money, mat,
                capital, agg, spot_ref, MV_EQUITY_T0,
                TARGET_COVERAGE, BAND_LOW, BAND_HIGH,
                max_notional_pct=MAX_NOTIONAL_PCT)

            for t in trades:
                d = t.to_dict()
                d['start_id'] = sid; d['start_date_bt'] = sd
                d['moneyness'] = money; d['maturity'] = mat
                all_detail.append(d); trades_this.append(d)

        agg.restore(agg_snapshot)  # FIX #2: clean for next sid

        if not trades_this: continue

        a = aggregate_single_start(trades_this, sid, sd, ed)
        for rec in a:
            if '_cum_pnl_dates' in rec and '_cum_pnl_values' in rec:
                m, mt = rec.get('moneyness'), rec.get('maturity')
                if m is not None and mt is not None:
                    cum_pnl[(m, mt, sid)] = (rec['_cum_pnl_dates'], rec['_cum_pnl_values'])
            all_agg.append(rec)

        if (sid+1) % 25 == 0 or sid == 0:
            el = time.time()-t0; eta = el/(sid+1)*(n_starts-sid-1)
            print(f"  {sid+1}/{n_starts} ({sd.date()}) — {el:.0f}s, ETA {eta:.0f}s  "
                  f"[MF={agg.marginal_factor('equity'):.3f}]")

    print(f"\nDone: {n_starts} starts in {time.time()-t0:.1f}s | {len(all_agg)} agg records")

    # ── 7. Distribution ───────────────────────────────────────────
    df_detail = pd.DataFrame(all_detail)
    df_agg = pd.DataFrame(all_agg)
    internal = [c for c in df_agg.columns if c.startswith('_')]
    df_agg_clean = df_agg.drop(columns=internal, errors='ignore')

    dist = compute_distribution(df_agg_clean, ['moneyness', 'maturity'])

    print(f"\n{'─'*70}\nDISTRIBUTION SUMMARY\n{'─'*70}")
    for section, cols in [
        ('PnL (ann.)', ['pnl_position_ann_mean','pnl_position_ann_median',
                        'pnl_position_ann_p5','pnl_position_ann_p95']),
        ('SCR per Unit', ['scr_gain_pu_iso_median','cost_pu_median',
                          'scr_net_pu_iso_median','efficiency_pu_iso_median']),
        ('Coverage', ['scr_coverage_mean_avg_median','n_adjustments_avg_median',
                      'premium_pct_mv_avg_median']),
    ]:
        avail = [c for c in cols if c in dist.columns]
        if avail:
            print(f"\n── {section} ──")
            print(dist[avail].to_string())

    # ── 8. Report ─────────────────────────────────────────────────
    os.makedirs(OUTPUT_FOLDER, exist_ok=True)
    print(f"\n{'='*70}\nGENERATING REPORT\n{'='*70}")

    generate_report(
        dist=dist, df_agg=df_agg_clean, df_detail=df_detail,
        spot_series=spot_series, sa_series=sa_series,
        cum_pnl_series=cum_pnl, candidate_starts=candidate_starts,
        start_date=START_DATE, end_date=END_DATE,
        window_days=window_days, n_starts=n_starts, window_years=window_years,
        focus_moneys=FOCUS_MONEYS, focus_mats=FOCUS_MATS, folder=OUTPUT_FOLDER,
        df_raw=df_raw,
        target_coverage=TARGET_COVERAGE, band_low=BAND_LOW, band_high=BAND_HIGH)

    # ── 9. Export CSV ─────────────────────────────────────────────
    dist.reset_index().to_csv(f'{OUTPUT_FOLDER}/distributional_summary.csv', index=False)
    df_agg_clean.to_csv(f'{OUTPUT_FOLDER}/agg_summary.csv', index=False)
    if not df_detail.empty:
        df_detail.to_csv(f'{OUTPUT_FOLDER}/trade_detail.csv', index=False)

    # ── Summary ───────────────────────────────────────────────────
    total = time.time() - t_total
    avg_cov = df_agg_clean['scr_coverage_mean_avg'].mean() \
              if 'scr_coverage_mean_avg' in df_agg_clean.columns else 0
    avg_adj = df_agg_clean['n_adjustments_avg'].mean() \
              if 'n_adjustments_avg' in df_agg_clean.columns else 0

    print(f"\n{'='*70}")
    print(f"COMPLETE — {total:.1f}s")
    print(f"  Report: {OUTPUT_FOLDER}/Rapport_Hedging.pdf")
    print(f"  MF equity: {agg.marginal_factor('equity'):.3f}")
    print(f"  Avg SCR coverage: {avg_cov:.1%}")
    print(f"  Avg adjustments/roll: {avg_adj:.1f}")
    if MAX_NOTIONAL_PCT is not None:
        cap_rate = df_detail['notional_capped'].mean() * 100 \
                   if 'notional_capped' in df_detail.columns and not df_detail.empty else 0
        print(f"  Notional cap: {MAX_NOTIONAL_PCT:.0%} of sleeve | "
              f"Binding in {cap_rate:.1f}% of rolls")
    print("=" * 70)


if __name__ == '__main__':
    main()

SCR-TARGET BACKTEST — Coverage 30% [25%–35%]
Roll mode: systematic
Notional cap: disabled (uncapped)
Loading: C:\\Users\\jules\\Desktop\\Hedging Framework - Thesis\\Input\\SPX_IV_Cleaned.csv
  2008-01-10 -> 2026-01-09 | 4529 days | 15 instruments

SCR Market Aggregation
──────────────────────────────────────────────────
  interest_rate          10,000,000   MF=0.592
  equity                 84,568,080   MF=0.985
  spread                  7,000,000   MF=0.790
  currency               12,000,000   MF=0.377
  ──────────────────────────────────────────────────
  SCR Market             99,286,526
  Sum (undiv)           113,568,080
  Divers. benefit        14,281,554

Instruments: 10
Window: 17.92y | Starts: 5
Building instrument-specific daily lookup...
  Built lookup for 10 instruments

RUNNING BACKTESTS
  1/5 (2008-01-10) — 11s, ETA 45s  [MF=0.985]

Done: 5 starts in 56.5s | 50 agg records

──────────────────────────────────────────────────────────────────────
DISTRIBUTION SUMMARY
──────